# Planet Ephemerides
## Reproducing Annual Values from the Astronomical Almanac

L.N. Fletcher

Based on IDL code provided by Data Reduction Manager (DRM) - B. Fisher & G. Orton.  Enormous thanks for the hints from Pat Fry.

In [85]:
import numpy as np
import matplotlib.pyplot as plt
import spiceypy as spice
import glob, re, os
import time


d2r=np.pi/180.
r2d=180./np.pi

## Read SPICE Kernels and Set Archinal (2009) J2000 Values

In [86]:
kernel_path = '/Users/lnf2/Data/generic_kernels/'

def find_spice_kernels(kernel_path):
    pcks  = sorted(glob.glob(kernel_path+"pck/*.tpc"))
    spks1 = sorted(glob.glob(kernel_path+"spk/planets/*.bsp"))
    #spks1 = sorted(glob.glob(kernel_path+"spk/de*.bsp"))
    spks2 = sorted(glob.glob(kernel_path+"spk/satellites/*.bsp"))
    fks   = sorted(glob.glob(kernel_path+"fk/planets/*.tf"))
    lsks  = sorted(glob.glob(kernel_path+"lsk/naif*.tls"))


    kernels = [pcks[-1], spks1[-1], *spks2, lsks[-1]]
    #kernels = [*pcks, *spks1, *spks2, *lsks, *fks]
    
    for kernel in kernels:
        #print(kernel)
        spice.furnsh(kernel)

find_spice_kernels(kernel_path)

In [87]:
# Transcription of Values from Archinal (2009) technical note
# Also the same as those in kernel pck0010.tpc
# Celest Mech Dyn Astr (2011) 109:101–135
# DOI 10.1007/s10569-010-9320-4

# JUPITER
# ---------------

# Rotation rates of three jovian longitude systems
wt3=870.536 #642 Extra sigfig in Almanac 2001, also in 2006 IAU report, but not in pck0010.tpc
wt2=870.270
wt1=877.900

#a1= North Pole RA
#a1=268.05773 # DRM Value
a0=268.056595 # Archinal Value and pck0010.tpc
arate=-0.006499

#d1= North Pole Dec
#d1=64.49712 # DRM Value
d0=64.495303 # Archinal Value and pck0010.tpc
drate=0.002413

# Argument of prime meridian for 2000 January 1st 12:00 TDB
w0_3=284.95
w0_2=43.3
w0_1=67.1

# SATURN
# ---------------
sat_wt3=810.7939024 
sat_w0_3=38.90 # Argument of prime meridian for 2000 January 1st 12:00 TDB
sat_a0=40.589
sat_arate=-0.036
sat_d0=83.537
sat_drate=-0.004

# URANUS
# ---------------
ura_wt3= -501.1600928
ura_w0_3=203.81 # Argument of prime meridian for 2000 January 1st 12:00 TDB
ura_a0=257.311
ura_arate=0.0
ura_d0= -15.175
ura_drate=0.0

# NEPTUNE
# ---------------
nep_wt3=536.312849
N=357.85 #  + 52.316T (assume T=0)
nep_w0_3=253.18-0.48*np.sin(N*d2r) # Argument of prime meridian for 2000 January 1st 12:00 TDB
nep_a0=299.36
nep_arate=0.70*np.sin(N*d2r)
nep_d0=43.4
nep_drate=-0.51*np.cos(N*d2r)



## Calculating Argument of Prime Meridian for Jupiter

In [69]:
# Note that the Archinal 2009 definitions are for 2000 Jan 1st 12:00 
# "Barycentric Dynamical Time (TDB)"

et1 = spice.str2et('2000-01-01TDB12:00')
nyrs=21

# Note that the Astronomical Almanac uses "TT" or "terrestrial time"
# which is the same as TDT "Terrestrial Dynamical Time"
for i in range(1980,2030,1):
    yr=i
    
    # Almanac gives positions at January 0, 0hr TT/TDT, which is the same as Dec 31T00:00
    dstring=str(yr)+'-12-31TDT00:00'
    #print(dstring)
    
    

    et2 = spice.str2et(dstring)
    days=(et2-et1)/(24.*60.*60.)

    [pos, ltime] = spice.spkpos( 'JUPITER', et2, 'J2000','NONE', 'EARTH')
    [r, ra, dec ] = spice.recrad(pos)
    dist = spice.vnorm( pos )
    delta = spice.convrt( dist, 'KM', 'AU' )
    a=ra*r2d
    d=dec*r2d

    utc=spice.et2datetime(et2)
    print('')
    print('JUPITER: ',yr+1,'January 0, 0h, TT - Ndays=','{0:8.2f}'.format(days))
    print(' Range (AU), RA, DEC: {0:8.6f}'.format(delta), '{0:8.6f}'.format(a), '{0:8.6f}'.format(d))

# Calculate the argument of prime meridian for a single day

    w1=w0_1 + days*wt1
    w2=w0_2 + days*wt2
    w3=w0_3 + days*wt3

    w1=np.mod(w1,360.)
    w2=np.mod(w2,360.)
    w3=np.mod(w3,360.)

    print(' Argument of PM 1/2/3: {:6.3f}'.format(w1),'{:6.3f}'.format(w2),'{:6.3f}'.format(w3))

    
    #From IDL code planetlcm.pro
    #Be=asind(-sind(d1)*sind(d)-cosd(d1)*cosd(d)*cosd(a1-a))  

    # Tcent is the interval in Julian centuries from J2000
    Tcent=days/36525.
    a1=a0+arate*Tcent
    d1=d0+drate*Tcent
    
    print(' N.Pole RA/DEC: {:6.3f}'.format(a1),'{:6.3f}'.format(d1))

    
    sd1=np.sin(d1*d2r)
    sd=np.sin(d*d2r)
    cd1=np.cos(d1*d2r)
    cd=np.cos(d*d2r)
    ca1a=np.cos((a1-a)*d2r)
    sa1a=np.sin((a1-a)*d2r)

    arg=(-sd1*sd - cd1*cd*ca1a)
    Be=np.arcsin(arg)

    print(" Sub-Earth Latitude (jovicentric):",'{:6.3f}'.format(Be*r2d))

    cBe=np.cos(Be)

    # Correction term for jovicentric longitude of sub-earth point
    # From IDL code planetlcm.pro
    #K=atan2d((-cosd(d1)*sind(d)+sind(d1)*cosd(d)*cosd(a1-a))/cosd(Be),(cosd(d)*sind(a1-a))/cosd(Be))

    arg1= (-cd1*sd+sd1*cd*ca1a)/cBe
    arg2= (cd*sa1a)/cBe

    #atan2d = double(atan(x1,x2)*57.2958), i.e.  the angle in degrees whose tangent is x2/x1.  

    K=np.arctan2(arg1,arg2)*r2d

    #print("Longitude Correction K:",K)

    # Argument of prime meridian at the time of observation andedated by light time.
    lcm3=w0_3+wt3*(days-.0057755*delta)-K
    lcm3=np.mod(lcm3,360.)

    lcm2=w0_2+wt2*(days-.0057755*delta)-K
    lcm2=np.mod(lcm2,360.)
    lcm1=w0_1+wt1*(days-.0057755*delta)-K
    lcm1=np.mod(lcm1,360.)    
    print(" Calculated LCMIII:",'{:6.3f}'.format(lcm3))
    print(" Calculated LCMII:",'{:6.3f}'.format(lcm2))
    print(" Calculated LCMI:",'{:6.3f}'.format(lcm1))


1981 January 0, 0h, TT - Ndays= -6940.50
 Range (AU), RA, DEC: 5.353012 189.428643 -2.647330
 Argument of PM 1/2/3:  2.150 14.365 209.842
 N.Pole RA/DEC: 268.058 64.495
 Sub-Earth Latitude (jovicentric): -2.471
 Calculated LCMIII: 171.518
 Calculated LCMII: 336.050
 Calculated LCMI: 323.599

1982 January 0, 0h, TT - Ndays= -6575.50
 Range (AU), RA, DEC: 5.818318 214.265630 -12.477405
 Argument of PM 1/2/3: 35.650 142.915 75.482
 N.Pole RA/DEC: 268.058 64.495
 Sub-Earth Latitude (jovicentric): -3.058
 Calculated LCMIII:  8.316
 Calculated LCMII: 75.758
 Calculated LCMI: 328.236

1983 January 0, 0h, TT - Ndays= -6210.50
 Range (AU), RA, DEC: 6.132344 239.277559 -19.602648
 Argument of PM 1/2/3: 69.150 271.465 301.122
 N.Pole RA/DEC: 268.058 64.495
 Sub-Earth Latitude (jovicentric): -3.023
 Calculated LCMIII: 207.302
 Calculated LCMII: 177.654
 Calculated LCMI: 335.069

1984 January 0, 0h, TT - Ndays= -5845.50
 Range (AU), RA, DEC: 6.239514 265.566592 -23.054520
 Argument of PM 1/2/3: 10

## Saturn Calculation

In [83]:
# Note that the Archinal 2009 definitions are for 2000 Jan 1st 12:00 
# "Barycentric Dynamical Time (TDB)"

et1 = spice.str2et('2000-01-01TDB12:00')
nyrs=21

# Note that the Astronomical Almanac uses "TT" or "terrestrial time"
# which is the same as TDT "Terrestrial Dynamical Time"
for i in range(1980,2030,1):
    yr=i
    
    # Almanac gives positions at January 0, 0hr TT/TDT, which is the same as Dec 31T00:00
    dstring=str(yr)+'-12-31TDT00:00'
    #print(dstring)
    
    

    et2 = spice.str2et(dstring)
    days=(et2-et1)/(24.*60.*60.)

    [pos, ltime] = spice.spkpos( 'SATURN', et2, 'J2000','NONE', 'EARTH')
    [r, ra, dec ] = spice.recrad(pos)
    dist = spice.vnorm( pos )
    delta = spice.convrt( dist, 'KM', 'AU' )
    a=ra*r2d
    d=dec*r2d

    utc=spice.et2datetime(et2)
    print('')
    print('SATURN:',yr+1,'January 0, 0h, TT - Ndays=','{0:8.2f}'.format(days))
    print(' Range (AU), RA, DEC: {0:8.6f}'.format(delta), '{0:8.6f}'.format(a), '{0:8.6f}'.format(d))

# Calculate the argument of prime meridian for a single day

    w3=sat_w0_3 + days*sat_wt3
    w3=np.mod(w3,360.)

    print(' Argument of PM 3: {:6.3f}'.format(w3))

    
    #From IDL code planetlcm.pro
    #Be=asind(-sind(d1)*sind(d)-cosd(d1)*cosd(d)*cosd(a1-a))  

    # Tcent is the interval in Julian centuries from J2000
    Tcent=days/36525.
    a1=sat_a0+sat_arate*Tcent
    d1=sat_d0+sat_drate*Tcent
    
    print(' N.Pole RA/DEC: {:6.3f}'.format(a1),'{:6.3f}'.format(d1))

    
    sd1=np.sin(d1*d2r)
    sd=np.sin(d*d2r)
    cd1=np.cos(d1*d2r)
    cd=np.cos(d*d2r)
    ca1a=np.cos((a1-a)*d2r)
    sa1a=np.sin((a1-a)*d2r)

    arg=(-sd1*sd - cd1*cd*ca1a)
    Be=np.arcsin(arg)

    print(" Sub-Earth Latitude (centric):",'{:6.3f}'.format(Be*r2d))

    cBe=np.cos(Be)

    # Correction term for jovicentric longitude of sub-earth point
    # From IDL code planetlcm.pro
    #K=atan2d((-cosd(d1)*sind(d)+sind(d1)*cosd(d)*cosd(a1-a))/cosd(Be),(cosd(d)*sind(a1-a))/cosd(Be))

    arg1= (-cd1*sd+sd1*cd*ca1a)/cBe
    arg2= (cd*sa1a)/cBe

    #atan2d = double(atan(x1,x2)*57.2958), i.e.  the angle in degrees whose tangent is x2/x1.  

    K=np.arctan2(arg1,arg2)*r2d

    #print("Longitude Correction K:",K)

    # Argument of prime meridian at the time of observation andedated by light time.
    lcm3=sat_w0_3+sat_wt3*(days-.0057755*delta)-K
    lcm3=np.mod(lcm3,360.)

     
    print(" Calculated LCMIII:",'{:6.3f}'.format(lcm3))



SATURN: 1981 January 0, 0h, TT - Ndays= -6940.50
 Range (AU), RA, DEC: 9.479438 189.873572 -1.698522
 Argument of PM 3: 243.820
 N.Pole RA/DEC: 40.596 83.538
 Sub-Earth Latitude (centric):  7.248
 Calculated LCMIII: 320.412

SATURN: 1982 January 0, 0h, TT - Ndays= -6575.50
 Range (AU), RA, DEC: 9.798021 200.923976 -6.192950
 Argument of PM 3: 263.595
 N.Pole RA/DEC: 40.595 83.538
 Sub-Earth Latitude (centric): 12.272
 Calculated LCMIII: 327.742

SATURN: 1983 January 0, 0h, TT - Ndays= -6210.50
 Range (AU), RA, DEC: 10.096320 211.691833 -10.290278
 Argument of PM 3: 283.369
 N.Pole RA/DEC: 40.595 83.538
 Sub-Earth Latitude (centric): 16.673
 Calculated LCMIII: 335.237

SATURN: 1984 January 0, 0h, TT - Ndays= -5845.50
 Range (AU), RA, DEC: 10.364172 222.324215 -13.899531
 Argument of PM 3: 303.144
 N.Pole RA/DEC: 40.595 83.538
 Sub-Earth Latitude (centric): 20.359
 Calculated LCMIII: 342.820

SATURN: 1985 January 0, 0h, TT - Ndays= -5479.50
 Range (AU), RA, DEC: 10.581888 233.024035 -16

## Uranus Calculation

In [89]:
# Note that the Archinal 2009 definitions are for 2000 Jan 1st 12:00 
# "Barycentric Dynamical Time (TDB)"

et1 = spice.str2et('2000-01-01TDB12:00')
nyrs=21

# Note that the Astronomical Almanac uses "TT" or "terrestrial time"
# which is the same as TDT "Terrestrial Dynamical Time"
for i in range(1980,2030,1):
    yr=i
    
    # Almanac gives positions at January 0, 0hr TT/TDT, which is the same as Dec 31T00:00
    dstring=str(yr)+'-12-31TDT00:00'
    #print(dstring)
    
    

    et2 = spice.str2et(dstring)
    days=(et2-et1)/(24.*60.*60.)

    [pos, ltime] = spice.spkpos( 'URANUS', et2, 'J2000','NONE', 'EARTH')
    [r, ra, dec ] = spice.recrad(pos)
    dist = spice.vnorm( pos )
    delta = spice.convrt( dist, 'KM', 'AU' )
    a=ra*r2d
    d=dec*r2d

    utc=spice.et2datetime(et2)
    print('')
    print('URANUS:',yr+1,'January 0, 0h, TT - Ndays=','{0:8.2f}'.format(days))
    print(' Range (AU), RA, DEC: {0:8.6f}'.format(delta), '{0:8.6f}'.format(a), '{0:8.6f}'.format(d))

# Calculate the argument of prime meridian for a single day

    w3=ura_w0_3 + days*ura_wt3
    w3=np.mod(w3,360.)

    print(' Argument of PM 3: {:6.3f}'.format(w3))

    
    #From IDL code planetlcm.pro
    #Be=asind(-sind(d1)*sind(d)-cosd(d1)*cosd(d)*cosd(a1-a))  

    # Tcent is the interval in Julian centuries from J2000
    Tcent=days/36525.
    a1=ura_a0+ura_arate*Tcent
    d1=ura_d0+ura_drate*Tcent
    
    print(' N.Pole RA/DEC: {:6.3f}'.format(a1),'{:6.3f}'.format(d1))

    
    sd1=np.sin(d1*d2r)
    sd=np.sin(d*d2r)
    cd1=np.cos(d1*d2r)
    cd=np.cos(d*d2r)
    ca1a=np.cos((a1-a)*d2r)
    sa1a=np.sin((a1-a)*d2r)

    arg=(-sd1*sd - cd1*cd*ca1a)
    Be=np.arcsin(arg)

    print(" Sub-Earth Latitude (centric):",'{:6.3f}'.format(Be*r2d))

    cBe=np.cos(Be)

    # Correction term for jovicentric longitude of sub-earth point
    # From IDL code planetlcm.pro
    #K=atan2d((-cosd(d1)*sind(d)+sind(d1)*cosd(d)*cosd(a1-a))/cosd(Be),(cosd(d)*sind(a1-a))/cosd(Be))

    arg1= (-cd1*sd+sd1*cd*ca1a)/cBe
    arg2= (cd*sa1a)/cBe

    #atan2d = double(atan(x1,x2)*57.2958), i.e.  the angle in degrees whose tangent is x2/x1.  

    K=np.arctan2(arg1,arg2)*r2d

    #print("Longitude Correction K:",K)

    # Argument of prime meridian at the time of observation andedated by light time.
    lcm3=K-(ura_w0_3+ura_wt3*(days-.0057755*delta))  # Note the change in sign as wt3<0
    lcm3=np.mod(lcm3,360.)

     
    print(" Calculated LCMIII:",'{:6.3f}'.format(lcm3))



URANUS: 1981 January 0, 0h, TT - Ndays= -6940.50
 Range (AU), RA, DEC: 19.518720 236.452949 -19.638623
 Argument of PM 3: 185.434
 N.Pole RA/DEC: 257.311 -15.175
 Sub-Earth Latitude (centric): -69.619
 Calculated LCMIII: 133.726

URANUS: 1982 January 0, 0h, TT - Ndays= -6575.50
 Range (AU), RA, DEC: 19.634215 240.883317 -20.575486
 Argument of PM 3: 142.000
 N.Pole RA/DEC: 257.311 -15.175
 Sub-Earth Latitude (centric): -73.471
 Calculated LCMIII: 182.637

URANUS: 1983 January 0, 0h, TT - Ndays= -6210.50
 Range (AU), RA, DEC: 19.745329 245.325841 -21.391633
 Argument of PM 3: 98.566
 N.Pole RA/DEC: 257.311 -15.175
 Sub-Earth Latitude (centric): -77.041
 Calculated LCMIII: 234.717

URANUS: 1984 January 0, 0h, TT - Ndays= -5845.50
 Range (AU), RA, DEC: 19.851319 249.775777 -22.084068
 Argument of PM 3: 55.132
 N.Pole RA/DEC: 257.311 -15.175
 Sub-Earth Latitude (centric): -80.069
 Calculated LCMIII: 292.614

URANUS: 1985 January 0, 0h, TT - Ndays= -5479.50
 Range (AU), RA, DEC: 19.944752 

## Calculation for Neptune

In [84]:
# Note that the Archinal 2009 definitions are for 2000 Jan 1st 12:00 
# "Barycentric Dynamical Time (TDB)"

et1 = spice.str2et('2000-01-01TDB12:00')
nyrs=21

# Note that the Astronomical Almanac uses "TT" or "terrestrial time"
# which is the same as TDT "Terrestrial Dynamical Time"
for i in range(1980,2030,1):
    yr=i
    
    # Almanac gives positions at January 0, 0hr TT/TDT, which is the same as Dec 31T00:00
    dstring=str(yr)+'-12-31TDT00:00'
    #print(dstring)
    
    

    et2 = spice.str2et(dstring)
    days=(et2-et1)/(24.*60.*60.)

    [pos, ltime] = spice.spkpos( 'NEPTUNE', et2, 'J2000','NONE', 'EARTH')
    [r, ra, dec ] = spice.recrad(pos)
    dist = spice.vnorm( pos )
    delta = spice.convrt( dist, 'KM', 'AU' )
    a=ra*r2d
    d=dec*r2d

    utc=spice.et2datetime(et2)
    print('')
    print('NEPTUNE:',yr+1,'January 0, 0h, TT - Ndays=','{0:8.2f}'.format(days))
    print(' Range (AU), RA, DEC: {0:8.6f}'.format(delta), '{0:8.6f}'.format(a), '{0:8.6f}'.format(d))

# Calculate the argument of prime meridian for a single day

    w3=nep_w0_3 + days*nep_wt3
    w3=np.mod(w3,360.)

    print(' Argument of PM 3: {:6.3f}'.format(w3))

    
    #From IDL code planetlcm.pro
    #Be=asind(-sind(d1)*sind(d)-cosd(d1)*cosd(d)*cosd(a1-a))  

    # Tcent is the interval in Julian centuries from J2000
    Tcent=days/36525.
    a1=nep_a0+nep_arate*Tcent
    d1=nep_d0+nep_drate*Tcent
    
    print(' N.Pole RA/DEC: {:6.3f}'.format(a1),'{:6.3f}'.format(d1))

    
    sd1=np.sin(d1*d2r)
    sd=np.sin(d*d2r)
    cd1=np.cos(d1*d2r)
    cd=np.cos(d*d2r)
    ca1a=np.cos((a1-a)*d2r)
    sa1a=np.sin((a1-a)*d2r)

    arg=(-sd1*sd - cd1*cd*ca1a)
    Be=np.arcsin(arg)

    print(" Sub-Earth Latitude (centric):",'{:6.3f}'.format(Be*r2d))

    cBe=np.cos(Be)

    # Correction term for jovicentric longitude of sub-earth point
    # From IDL code planetlcm.pro
    #K=atan2d((-cosd(d1)*sind(d)+sind(d1)*cosd(d)*cosd(a1-a))/cosd(Be),(cosd(d)*sind(a1-a))/cosd(Be))

    arg1= (-cd1*sd+sd1*cd*ca1a)/cBe
    arg2= (cd*sa1a)/cBe

    #atan2d = double(atan(x1,x2)*57.2958), i.e.  the angle in degrees whose tangent is x2/x1.  

    K=np.arctan2(arg1,arg2)*r2d

    #print("Longitude Correction K:",K)

    # Argument of prime meridian at the time of observation andedated by light time.
    lcm3=nep_w0_3+nep_wt3*(days-.0057755*delta)-K
    lcm3=np.mod(lcm3,360.)

     
    print(" Calculated LCMIII:",'{:6.3f}'.format(lcm3))



NEPTUNE: 1981 January 0, 0h, TT - Ndays= -6940.50
 Range (AU), RA, DEC: 31.219080 262.764985 -21.974910
 Argument of PM 3: 13.870
 N.Pole RA/DEC: 299.365 43.497
 Sub-Earth Latitude (centric): -16.409
 Calculated LCMIII: 222.366

NEPTUNE: 1982 January 0, 0h, TT - Ndays= -6575.50
 Range (AU), RA, DEC: 31.226635 265.007142 -22.106871
 Argument of PM 3: 288.059
 N.Pole RA/DEC: 299.365 43.492
 Sub-Earth Latitude (centric): -17.209
 Calculated LCMIII: 134.523

NEPTUNE: 1983 January 0, 0h, TT - Ndays= -6210.50
 Range (AU), RA, DEC: 31.231626 267.254869 -22.208274
 Argument of PM 3: 202.249
 N.Pole RA/DEC: 299.364 43.487
 Sub-Earth Latitude (centric): -17.990
 Calculated LCMIII: 46.668

NEPTUNE: 1984 January 0, 0h, TT - Ndays= -5845.50
 Range (AU), RA, DEC: 31.233773 269.507069 -22.278840
 Argument of PM 3: 116.439
 N.Pole RA/DEC: 299.364 43.482
 Sub-Earth Latitude (centric): -18.750
 Calculated LCMIII: 318.804

NEPTUNE: 1985 January 0, 0h, TT - Ndays= -5479.50
 Range (AU), RA, DEC: 31.230413